In [3]:
import tensorflow as tf
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import helper_functions as hp
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch import nn
from torch.optim import Adam
from torchvision.transforms import  Compose, ToTensor, Resize, RandomHorizontalFlip, RandomRotation, ColorJitter, RandomResizedCrop, Lambda
from PIL import Image
import matplotlib.pyplot as plt
from torchsummary import summary
import torchmetrics
from lightning.pytorch.loggers import WandbLogger
import wandb
import random
from datetime import datetime
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

2025-01-30 15:43:55.951429: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-30 15:43:55.959554: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738248235.969343  158462 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738248235.972082  158462 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 15:43:55.982160: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
num_workers = 0

device = torch.device("cpu")

Set start method to fork and device to metal

## Linux

In [5]:
print("CUDA verfügbar:", torch.cuda.is_available())
print("CUDA-Version:", torch.version.cuda)
print("GPU-Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Keine GPU gefunden")

if torch.cuda.is_available():
    device = torch.device("cuda")
    num_workers = 2


CUDA verfügbar: True
CUDA-Version: 12.4
GPU-Name: NVIDIA GeForce RTX 3070 Ti


## Load dataset

In [6]:
# Dataset Path
DATASET_PATHS = ["./generated_images"]
PRICES_FILE_PATH = "../Dataset/prices.txt"
IMAGE_SIZE = (200, 200)
BATCH_SIZE = 32
SEED = 42

In [7]:
num_classes = 3

In [11]:
class ImagePriceDataset(Dataset):
    def __init__(self, dataset_paths, transform=None):
        self.files = []
        for dataset_path in dataset_paths:
            if not os.path.exists(dataset_path):
                raise ValueError(f"Path {dataset_path} doesn't exist.")
            for f in os.listdir(dataset_path):
                if f.endswith(".png"):
                    self.files.append((f, dataset_path))
        
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_name, dataset_path = self.files[idx]
        image, label = hp.load_generated_images(file_name, dataset_path, IMAGE_SIZE, num_classes)

        # Wenn das Bild im float32 Format vorliegt, skaliere es und wandle es in uint8 um
        if image.dtype == np.float32:
            image = (image * 255).astype(np.uint8)  # Skalieren auf [0, 255] und in uint8 umwandeln

        # Konvertiere das numpy-Array in ein PIL-Image
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)
    
transform_test = Compose([
    ToTensor()                         # Konvertiere das Bild zu einem Tensor
])

In [12]:


dataset = ImagePriceDataset(DATASET_PATHS, transform=transform_test)

test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)

In [16]:
class TrainModel(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-3, optimizer=None):
        super().__init__()
        
        self.learning_rate = learning_rate
        self.model = model
        self.criterion = nn.MSELoss()
        self.mae = torchmetrics.MeanAbsoluteError()
        self.mse = torchmetrics.MeanSquaredError()
        self.optimizer = optimizer
        self.threshold = 0.5
        self.save_hyperparameters() #ignore=['model']

    def forward(self, x):
        return self.model(x)

    def step(self, batch, stage):
        x, y = batch
        y_pred = self(x)  # Vorhersagen
        # Hauptverlust berechnen (MSE auf allen Labels)
        loss = self.criterion(y_pred, y)

        # MAE und MSE der Hauptlabels
        mae = self.mae(y_pred, y)
        mse = self.mse(y_pred, y)

        # Logs für Progress-Bar und Training/Validierung/Test
        self.log(f"{stage}_loss", loss, prog_bar=True, on_epoch=True)
        self.log(f"{stage}_mae", mae, prog_bar=True, on_epoch=True)
        self.log(f"{stage}_mse", mse, prog_bar=True, on_epoch=True)

        return loss

    def training_step(self, batch, batch_idx):
        return self.step(batch, stage="train")

    def validation_step(self, batch, batch_idx):
        return self.step(batch, stage="val")

    def test_step(self, batch, batch_idx):
        return self.step(batch, stage="test")

    def configure_optimizers(self):
        if self.optimizer is None:
            self.optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        
        # Scheduler erstellen
        scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=5, gamma=0.965)
        return [self.optimizer], [scheduler]


In [18]:
import helper_functions as hp

model = TrainModel.load_from_checkpoint("./checkpoints/classifier/generated_images_Efficientnet-classifier-generated_images_30Jan-15:13:41.ckpt")
model = model.model
model.eval()
# Assuming you have a PyTorch model instance `model`
# and a dummy input tensor that matches your model's input size
dummy_input = torch.randn(1, 3, 200, 200)  # Adjust dimensions as needed

# Export the model to ONNX
torch.onnx.export(model.to("cpu"), dummy_input, "quantization/b0-onGeneratedDB.onnx", verbose=False)


In [1]:
from onnx2tf import convert

# Pfad zum ONNX-Modell
onnx_model_path = "quantization/b0-onGeneratedDB.onnx"

# Ausgabeordner für das TensorFlow-Modell
output_folder = "quantization/b0-onGeneratedDB"

# Konvertierung durchführen
convert(
    input_onnx_file_path=onnx_model_path,  # Pfad zum ONNX-Modell
    output_folder_path=output_folder,  # Ordner für das konvertierte TensorFlow-Modell
    copy_onnx_input_output_names_to_tflite=True,  # Optional: ONNX-Ein-/Ausgabebenennungen behalten
)


E0000 00:00:1738248547.156593  159862 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738248547.159690  159862 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



Model optimizing started ============================================================
Traceback (most recent call last):
  File "/home/torgeschwark/miniconda3/envs/ichbinverzweifelt/lib/python3.10/site-packages/onnx2tf/onnx2tf.py", line 652, in convert
    result = subprocess.check_output(
  File "/home/torgeschwark/miniconda3/envs/ichbinverzweifelt/lib/python3.10/subprocess.py", line 421, in check_output
    return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
  File "/home/torgeschwark/miniconda3/envs/ichbinverzweifelt/lib/python3.10/subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "/home/torgeschwark/miniconda3/envs/ichbinverzweifelt/lib/python3.10/subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/home/torgeschwark/miniconda3/envs/ichbinverzweifelt/lib/python3.10/subprocess.py", line 1863, in _execute_child
    raise child_exception_type(errno_num, err_msg, err_filename

I0000 00:00:1738248569.231421  159862 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1738248569.231572  159862 single_machine.cc:361] Starting new session
W0000 00:00:1738248569.537725  159862 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1738248569.537758  159862 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1738248569.947600  159862 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1738248569.947742  159862 single_machine.cc:361] Starting new session


wget https://github.com/PINTO0309/onnx2tf/releases/download/1.16.31/flatc.tar.gz && tar -zxvf flatc.tar.gz && sudo chmod +x flatc && sudo mv flatc /usr/bin/
Float32 tflite output complete!


W0000 00:00:1738248570.209417  159862 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1738248570.209463  159862 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.


wget https://github.com/PINTO0309/onnx2tf/releases/download/1.16.31/flatc.tar.gz && tar -zxvf flatc.tar.gz && sudo chmod +x flatc && sudo mv flatc /usr/bin/
Float16 tflite output complete!
